# Calculating mAP

In [7]:
from ultralytics import YOLO
from PIL import Image
import numpy as np
import pandas as pd
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import transforms as T
import torchvision
from super_gradients.training import models

In [8]:
def load_and_initialize_model_faster_rcnn(model_path):
    # Define your model architecture
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    num_classes = 2  # Modify this based on your dataset
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Load the saved model checkpoint
    checkpoint = torch.load(model_path,map_location=torch.device('cpu'))

    # Load the model state and optimizer state
    model.load_state_dict(checkpoint['model_state_dict'])

    # Move the model to the GPU
    device = torch.device('cpu')
    model.to(device)

    # Set the model to evaluation mode
    model.eval()

    return model

def load_and_initialize_model_yolo_v8(model_path):
  # Load a pretrained YOLOv8n model
  model = YOLO(model_path)
  return model

def load_and_initialize_model_yolo_nas(path):
    
    modelyoloNas = models.get('yolo_nas_l',
                        num_classes=1,
                        checkpoint_path=path)
    return modelyoloNas

def load_model():

    model_path_rcnn="models/single_class_models/Singleclass_Faster_RCNN_125.pth"
    model_path_yolo="models/single_class_models/YoloV8.onnx"
    model_path_yolonas = "models/single_class_models/YoloNAS.pth"
    model_rcnn = load_and_initialize_model_faster_rcnn(model_path_rcnn)
    model_yolo = load_and_initialize_model_yolo_v8(model_path_yolo)
    model_yolonas = load_and_initialize_model_yolo_nas(model_path_yolonas)
    return model_rcnn,model_yolo,model_yolonas

In [9]:
def load_annotations_csv():

    annotations = pd.read_csv("/kaggle/input/plastics-in-water-bodies-with-annotations/River data/Annotations/test_annotations_.csv")
    image_ids = annotations['image_id'].unique()

    ground_truths = {}
    for image_id in image_ids:
        image_annotations = annotations[annotations['image_id'] == image_id]
        boxes = image_annotations[['x1', 'y1', 'x2', 'y2']].values
        labels = torch.zeros(len(boxes), dtype=torch.int64)  # Assuming there is only one class (label 0)
        ground_truths[image_id] = {'boxes': boxes,
                                   'labels': np.array(labels)}

    return ground_truths

In [10]:
def predict_on_single_image_rcnn(modelx, image_path, score_threshold=0.5):

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    img = Image.open(image_path).convert('RGB')
    transform = T.Compose([T.ToTensor()])
    img_tensor = transform(img).unsqueeze(0).to(device)
    modelx.eval()
    with torch.no_grad():
        output = modelx(img_tensor)

    boxes = output[0]['boxes'].cpu().detach().numpy()
    scores = output[0]['scores'].cpu().detach().numpy()
    labels = output[0]['labels'].cpu().detach().numpy()
#     print(output)
    # Filter out boxes with scores less than the threshold
    mask = scores >= score_threshold
    filtered_boxes = boxes[mask]
    filtered_scores = scores[mask]
    filtered_labels = labels[mask]

    return {'boxes': filtered_boxes, 'scores': filtered_scores, 'labels': filtered_labels}

def predict_on_single_image_yolo(modelx ,image_path):
  results = modelx(source=image_path,conf=0.1,iou=0.2)
  for result in results:
    boxes = result.boxes.xyxy
    scores = result.boxes.conf
    labels = result.boxes.cls
    boxes = boxes.cpu().numpy()
    scores = scores.cpu().numpy()
    labels = labels.cpu().numpy()
  results_dict = {'boxes': boxes, 'scores': scores, 'labels': labels}
  return results_dict

def predict_on_single_image_yolonas(modelx ,image_path):
    # yolonas = best_model.predict(image_path,conf=0.4,iou=0.7).show()
    yolonas = modelx.predict(image_path,conf=0.45,iou=0.7)

    for result in yolonas:
        yolovnas_preds = result.prediction.bboxes_xyxy
        nas_score = result.prediction.confidence
        labels = yolonas[0].prediction.labels
        
    results_dict = {'boxes': yolovnas_preds, 'scores': nas_score, 'labels': labels}
    return results_dict

In [11]:
# Apply non-maximum suppression to the concatenated predictions
def non_max_suppression(detections, threshold=0.5):
    # If there are no detections, return an empty list
    if len(detections) == 0:
        return []

    # Sort detections by their y_max coordinates (bottom of the bounding box)
    sorted_indices = np.argsort(detections[:, 3])[::-1]
    detections = detections[sorted_indices]

    # Initialize the list of selected detections
    selected_detections = []

    while len(detections) > 0:
        # Select the detection with the highest y_max (bottom) coordinate
        current_detection = detections[0]
        selected_detections.append(current_detection)

        # Calculate the overlap area with other detections
        x1_int = np.maximum(detections[1:, 0], current_detection[0])
        y1_int = np.maximum(detections[1:, 1], current_detection[1])
        x2_int = np.minimum(detections[1:, 2], current_detection[2])
        y2_int = np.minimum(detections[1:, 3], current_detection[3])

        w_int = np.maximum(0, x2_int - x1_int + 1)
        h_int = np.maximum(0, y2_int - y1_int + 1)
        area_int = w_int * h_int

        iou = area_int / ((current_detection[2] - current_detection[0] + 1) *
                          (current_detection[3] - current_detection[1] + 1) +
                          (detections[1:, 2] - detections[1:, 0] + 1) *
                          (detections[1:, 3] - detections[1:, 1] + 1) - area_int)

        # Filter out detections with IoU above the threshold
        overlapping_indices = np.where(iou <= threshold)[0]
        detections = detections[overlapping_indices + 1]

    return np.array(selected_detections)

In [12]:
def calculate_mAP_for_ensemble(test_image_folder):
    # Load models and annotations
    model1, model2 ,model3= load_model()
    ground_truths = load_annotations_csv()
    test_results = []
    ensemble_predictions = None
    for image_id in ground_truths.keys():
        
        image_path = f"{test_image_folder}/{image_id}.jpg"

        predictions_model1 = predict_on_single_image_rcnn(model1, image_path)
        predictions_model2 = predict_on_single_image_yolo(model2, image_path)
        predictions_model3 = predict_on_single_image_yolonas(model3, image_path)

        # Concatenate the predictions from both models
        all_predictions = np.vstack((predictions_model1['boxes'], predictions_model2['boxes'],predictions_model3['boxes']))
        ensemble_predictions = non_max_suppression(all_predictions)
        test_results.append((ensemble_predictions, ground_truths[image_id]))
    return test_results

# Example usage
test_image_folder = r"/kaggle/input/plastics-in-water-bodies-with-annotations/River data/Test"
mAP_ensemble = calculate_mAP_for_ensemble(test_image_folder)

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.
[2023-09-05 11:53:16] INFO - checkpoint_utils.py - Successfully loaded model weights from models/single_class_models/YoloNAS.pth EMA checkpoint.


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/plastics-in-water-bodies-with-annotations/River data/Annotations/test_annotations_.csv'

: 

In [ ]:
print(len(mAP_ensemble))

30


In [ ]:
import numpy as np
from sklearn.metrics import average_precision_score

def calculate_mAP(predictions_list):
    num_classes = 1
    thresh = 0.225
    mAP_sum = 0.0
    num_images = len(predictions_list)

    for i in range(num_images):
        predictions, gt_boxes_i = predictions_list[i]

        # Calculate Average Precision for each class
        ap_sum = 0.0
        for c in range(num_classes):
            pred_boxes_c = predictions
            gt_boxes_c = gt_boxes_i['boxes'][np.where(gt_boxes_i['labels'] == c)[0]]

            if len(gt_boxes_c) == 0 or len(pred_boxes_c) == 0:
                continue

            iou_matrix = calculate_iou(pred_boxes_c, gt_boxes_c)
            tp_indices = np.where(iou_matrix >= thresh)[0]
            num_true_positives = len(tp_indices)
            ap_sum += num_true_positives / len(gt_boxes_c)

        ap = ap_sum / num_classes
        mAP_sum += ap

    mean_ap = mAP_sum / num_images
    return mean_ap


def calculate_iou(boxes1, boxes2):
    iou_matrix = np.zeros((len(boxes1), len(boxes2)))

    for i in range(len(boxes1)):
        for j in range(len(boxes2)):
            x1 = max(boxes1[i, 0], boxes2[j, 0])
            y1 = max(boxes1[i, 1], boxes2[j, 1])
            x2 = min(boxes1[i, 2], boxes2[j, 2])
            y2 = min(boxes1[i, 3], boxes2[j, 3])

            intersection = max(0, x2 - x1) * max(0, y2 - y1)
            area_boxes1 = (boxes1[i, 2] - boxes1[i, 0]) * (boxes1[i, 3] - boxes1[i, 1])
            area_boxes2 = (boxes2[j, 2] - boxes2[j, 0]) * (boxes2[j, 3] - boxes2[j, 1])
            union = area_boxes1 + area_boxes2 - intersection

            iou_matrix[i, j] = intersection / (union + np.finfo(np.float64).eps)

    return iou_matrix

mAP = calculate_mAP(mAP_ensemble)
print("Mean Average Precision (mAP):", mAP)

Mean Average Precision (mAP): 0.9419797270439686
